# Data Imputation with Last Seen Value

In [2]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics

from zoo.chronos.preprocessing.impute.LastFill import LastFill

In [3]:
raw_df = pd.read_csv("data/data.csv")

In [4]:
df = pd.DataFrame(pd.to_datetime(raw_df.StartTime))

In [5]:
raw_df.AvgRate.str[-4:].unique()
# Unify AvgRate value
df['AvgRate'] = raw_df.AvgRate.apply(lambda x: float(x[:-4]) if x.endswith("Mbps") else float(x[:-4]) * 1000)
df["total"] = raw_df["total"]
df.set_index("StartTime", inplace=True)

In [6]:
df.head()

,AvgRate,total
StartTime,,
2018-01-01 00:00:00,306.23,275605455598
2018-01-01 02:00:00,285.03,256527692256
2018-01-01 04:00:00,247.39,222652190823
2018-01-01 06:00:00,211.55,190396029658
2018-01-01 08:00:00,234.82,211340468977


In [7]:
sampled_df = df["2019-01-01 00:00:00":"2019-06-30 23:00:00"]

In [8]:
full_idx = pd.date_range(start=sampled_df.index.min(), end=sampled_df.index.max(), freq='2H')
sampled_df = df.reindex(full_idx)
print("no. of n/a values:")
print(sampled_df.isna().sum())

no. of n/a values:
AvgRate    0
total      0
dtype: int64


In [9]:
sampled_df

,AvgRate,total
2019-01-01 00:00:00,271.53,244373500495
2019-01-01 02:00:00,208.49,187638154842
2019-01-01 04:00:00,175.88,158293382443
2019-01-01 06:00:00,229.20,206284366389
2019-01-01 08:00:00,264.95,238459494810
...,...,...
2019-06-30 14:00:00,618.90,557013671978
2019-06-30 16:00:00,533.31,479975450009
2019-06-30 18:00:00,475.86,428273046817
2019-06-30 20:00:00,448.12,403312442571


In [10]:
last_fill = LastFill()

In [11]:
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='2H')
df1 = df.reindex(full_idx)
print("no. of n/a values:")
print(df1.isna().sum())

no. of n/a values:
AvgRate    3
total      3
dtype: int64


In [12]:
filled_df = last_fill.impute(df)

In [13]:
full_idx = pd.date_range(start=df.index.min(), end=df.index.max(), freq='2H')
fill_df = filled_df.reindex(full_idx)
print("no. of n/a values:")
print(filled_df.isna().sum())

no. of n/a values:
AvgRate    0
total      0
dtype: int64


In [14]:
mse_10 = last_fill.evaluate(sampled_df, 0.1)

In [15]:
mse_10

[2060.001521362799, 2.3649853916972995e+21]

In [16]:
mse_50 = last_fill.evaluate(sampled_df, 0.5)

In [17]:
mse_50

[29992.315269981584, 2.183801163114765e+22]